# z3

In [1]:
import re
import numpy as np

word2tag = dict()
tag2word = dict()
def stringNorm(sent, num=False):
    regex = re.compile(f'[,\.!?:;\'{"0-9" if not num else ""}\*\-“…\(\)„”—»«–––=\[\]’]')
    return regex.sub('',sent.lower())

with open("data/supertags.txt") as tags:
    for line in tags:
        word, tag = stringNorm(line, num=True).split()
        word2tag[word] = tag
        if tag in tag2word:
            tag2word[tag].append(word)
        else:
            tag2word[tag] = [word]

In [10]:
def bigrams2unigrams(bigrams):
    return {w1: sum([float(bigrams[w1][w2]) for w2 in bigrams[w1]])/2 for w1 in bigrams}
unigrams = bigrams2unigrams(bigrams)

In [11]:
def getProb(word):
    if word in unigrams:
        return unigrams[word]*10
    return 0.001

def getRandWord(words):
    probs = np.array([getProb(x) for x in words])
    probs = probs / np.sum(probs)
    return str(np.random.choice(words, 1, p=probs)[0])

def getRandomSentence(model):

    sent = stringNorm(model).split()
    sentCodes = [(word2tag[x] if x in word2tag 
                  else (print(f"***Nie znaleziono takiego słowa: {x}***"),
                        word2tag[('^' + x)[-3:]])[1]) 
                 for x in sent]
    altWords = [tag2word[x] for x in sentCodes]
    newSentence = [getRandWord(x) for x in altWords]
    return ' '.join(newSentence)

In [12]:
sentenceList = [
    "Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.",
    "Zbyt zabawne było powstrzymywanie się od śmiechu żeby to zrobić",
    "Dawno nie piła tak dobrego, świeżego mleka",
    "Niestety komputer postanowił odmówić posłuszeństwa",
    "Mama Darka Czuje Ból Na pewno Musi Wypoczywać. ",
    "Idę do sklepu kupić nowe spodnie",
    "Kyeloger wywalił renderowanie filmów"
]
                

In [14]:
for model in sentenceList:
    print(model,'\n',getRandomSentence(model),'\n')

Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę. 
 bezpłatny pan zachował za nierdzewnej rzece oraz związaną infekcję 

Zbyt zabawne było powstrzymywanie się od śmiechu żeby to zrobić 
 przykład polskie zostało podjęcie do z sojuszu się to przeczytać 

Dawno nie piła tak dobrego, świeżego mleka 
 w nie piła tutaj międzynarodowego całego rozwiązania 

Niestety komputer postanowił odmówić posłuszeństwa 
 w wkład przyjął nacisnąć doradztwa 

Mama Darka Czuje Ból Na pewno Musi Wypoczywać.  
 dziewczyna kontrolera chodzi ból a a musi sięgać 

Idę do sklepu kupić nowe spodnie 
 ciążę o urzędu uzyskać konkretne spodnie 

***Nie znaleziono takiego słowa: kyeloger***
***Nie znaleziono takiego słowa: renderowanie***
Kyeloger wywalił renderowanie filmów 
 projekt otrzymał nie murów 



# z4

In [5]:
import os
os.sys.path.append('../1/')
from z2 import loader
bigrams = loader('../1/poleval_2grams.txt', cut = 10)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000


In [6]:
def getProb2(word, prev):
    if word in bigrams[prev]:
#         print('aa: ', bigrams[prev][word])
        return bigrams[prev][word]
    return 0.001

def getRandWord2(words, prev):
    probs = np.array([getProb2(x, prev) for x in words]).astype(float)
    probs = probs / np.sum(probs)
    return str(np.random.choice(words, 1, p=probs)[0])

In [7]:
def getRandomSentenceBi(model):
    sent = stringNorm(model).split()
    sentCodes = [(word2tag[x] if x in word2tag 
                  else (print(f"***Nie znaleziono takiego słowa: {x}***"),
                        word2tag[('^' + x)[-3:]])[1]) 
                 for x in sent]
    altWords = [tag2word[x] for x in sentCodes]
    
    i = 0
    newSentence = []
    
    while i < len(altWords):
        grammWords = set(altWords[i])
        bigramWords = {y for y in bigrams}
        goodWords = grammWords.intersection(bigramWords)
        x = getRandWord(list(goodWords))
        newSentence.append(x)
        i+=1
        while x in bigrams and i < len(altWords):
            grammWords = set(altWords[i])
            bigramWords = {y for y in bigrams[x]}
            goodWords = grammWords.intersection(bigramWords)
            if len(goodWords) <1:
                break
            x = getRandWord2(list(goodWords), x)
            newSentence.append(x)
            i+=1

        newSentence.append('|')
        
    return ' '.join(newSentence[:-1])

In [15]:
for model in sentenceList:
    print(model,'\n',getRandomSentenceBi(model),'\n')

Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę. 
 ważny partner | poinformował o niewystarczającej liczbie około | kierowaną | izbę 

Zbyt zabawne było powstrzymywanie się od śmiechu żeby to zrobić 
 spadek | średnie | zostało rozporządzenie z i stylu w to ograniczyć 

Dawno nie piła tak dobrego, świeżego mleka 
 do nie piła | czy innego miejscowego środowiska 

Niestety komputer postanowił odmówić posłuszeństwa 
 za akt | przeniósł | powiedzieć | dziennikarstwa 

Mama Darka Czuje Ból Na pewno Musi Wypoczywać.  
 ustawa | trenera | chodzi | ból w znacznie | musi podlegać 

Idę do sklepu kupić nowe spodnie 
 idę do rozłamu | stwierdzić | odwoławcze | spodnie 

***Nie znaleziono takiego słowa: kyeloger***
***Nie znaleziono takiego słowa: renderowanie***
Kyeloger wywalił renderowanie filmów 
 system sprawdził | nie milionów 

